In [ ]:
import os
import pandas as pd
import re
import numpy as np

In [ ]:
# Load old salaries
pwd = os.getcwd()
old_salaries = os.path.join(pwd, 'Resources', 'salaries', 'salaries_1985to2018.csv')
old_salaries_df = pd.read_csv(old_salaries)

In [ ]:
# load new salaries
new_salaries = os.path.join(pwd, 'Resources', 'salaries', '1999-2022 Salaries.csv')
new_salaries_df = pd.read_csv(new_salaries)

In [ ]:
# load player data
players = os.path.join(pwd, 'Resources', 'salaries', 'players.csv')
players_df = pd.read_csv(players)

In [ ]:
# get old salaries row count
print(old_salaries_df.shape)
old_salaries_df.head()

In [ ]:
# check to see what needs to be combined
new_salaries_df.head()

In [ ]:
# check the column names
players_df.head()

In [ ]:
# Clean the old salaries so we can merge on the player id
# Drop the league column
old_salaries_df.drop(columns = 'league', inplace = True)

# Change the columns so that everything will be lowered cased and reorganized 
# we want to see player id, team, salary, season info
old_salaries_df.rename(columns = {"player_id": "_id"}, inplace=True)
new_col = ['_id', 'team', 'salary', 'season', 'season_end', 'season_start']
old_salaries_df = old_salaries_df[new_col]

In [ ]:
# check old salaries columns
old_salaries_df

In [ ]:
# merge players and old salaries
merged_df = pd.merge(players_df, old_salaries_df, how= "right", on="_id")

In [ ]:
print(merged_df.shape)
merged_df.sample(10)

In [ ]:
# check to see if columns merged successfuly 
merged_df.columns.tolist()

In [ ]:
# remove unnecessary data
col_removed = ['_id', 'birthDate', 'birthPlace', 'college', 'highSchool', 'weight', 'career_FG%', 'career_AST', 'career_FG3%', 
             'career_FT%', 'career_G',  'career_PER', 'career_PTS', 'career_TRB', 'career_WS', 'career_eFG%', 'shoots']
merged_df.drop(columns=col_removed, inplace = True)
merged_df

In [ ]:
merged_df.columns.tolist()

In [ ]:
# reorder columns for readability
col_reorder = ['season', 'season_end', 'season_start', 'name', 'team', 'position', 'salary', 'height','draft_pick',
               'draft_round','draft_team','draft_year']
merged_df = merged_df[col_reorder]

In [ ]:
print(merged_df.shape)
merged_df.head()

In [ ]:
merged_df.dtypes

In [ ]:
Drop Rank column
new_salaries_df.drop(columns= 'RK', inplace = True)

In [ ]:
new_salaries_df

In [ ]:
# remove the commas and the dollar sign in the salary column
new_salaries_df['SALARY'] = new_salaries_df['SALARY'].str.replace(',', '').str.replace('$', '').astype(int)
new_salaries_df

In [ ]:
# we want to split the names into last name first name
split_df = new_salaries_df['NAME'].str.split(pat=",", expand = True)

In [ ]:
# split name and position 
split_df.columns= ['name', 'position']

In [ ]:
split_df.head()

In [ ]:
# look for positions to map
split_df['position'].unique()

In [ ]:
# check positions to map
merged_df['position'].unique()

In [ ]:
# # new function that changes the columns name. 
# def clean_pos(player_position):
#     players_position = dict(player_position)
#     def change_column_name(old_name, new_name):
#         if old_name in play_positions:
#             play_position[new_name] = play_position.pop(old_name)
#         change_column_name('PG', 'Point Guard')
#         change_column_name('SG', 'Shooting Guard')
#         change_column_name('PF', 'Power Forward')
#         change_column_name('SF', 'Small Forward')
#         change_column_name('C', 'Center')
#         change_column_name('G', 'Shooting Guard and Point Guard')
#         change_column_name('F', 'Small Forward and Power Forward')
#         change_column_name('GF', 'Small Forward and Shooting Guard')

#         return positions

In [ ]:
# players_position = dict(split_df)

In [ ]:
# for players in players_position:
#     clean_pos(split_df)

In [ ]:
# map the new data frame with the old so they share the same position naming
positions_dict = {' PG':'Point Guard',' SG':'Shooting Guard',' PF':'Power Forward',
                  ' SF':'Small Forward', ' C':'Center', ' G':'Shooting Guard and Point Guard',
                 ' F':'Small Forward and Power Forward', ' GF':'Small Forward and Shooting Guard'}

In [ ]:
# https://stackoverflow.com/questions/57873297/pandas-to-replace-value-using-map-function
# using map function to change the positions
split_df['position'] = split_df['position'].map(positions_dict)

In [ ]:
# check to see if it is successful
split_df.head(20)

In [ ]:
# split the start and end season for easier filtering
dates_df = new_salaries_df['season'].str.split(pat="-", expand = True).astype(int)
dates_df.columns=['season_start', 'season_end']
dates_df.head()

In [ ]:
# merge new salaries with preprocessed data
new_salaries_df = new_salaries_df.merge(split_df, left_index=True, right_index= True)
new_salaries_df = new_salaries_df.drop(['NAME'], axis =1)
new_salaries_df.head()

In [ ]:
# check new_salaries df to see if it is merged
new_salaries_df = pd.concat([new_salaries_df, dates_df], axis=1)
new_salaries_df.head()

In [ ]:
# reorder columns and change to lower case for case issues
new_salaries_df = new_salaries_df[['season', 'season_end', 'season_start', 'name', 'TEAM', 'position', 'SALARY']]
new_salaries_df.rename(columns = {"TEAM": "team", "SALARY":"salary"}, inplace=True)
new_salaries_df.head()

In [ ]:
# check datatypes
new_salaries_df.dtypes

In [ ]:
# find the salary needed to merge
current_salaries_df = new_salaries_df.loc[new_salaries_df['season_start'] >= 2018]
current_salaries_df

In [ ]:
# check merged df
merged_df

In [ ]:
# drop unnecessary columns
merged_df = merged_df.drop(columns = ['height', 'draft_pick', 'draft_round', 'draft_team', 'draft_year'], axis = 1)
merged_df

In [ ]:
# merge the new and old into one data frame
final_salaries = merged_df.append(current_salaries_df, ignore_index=True, sort=False)
final_salaries

In [ ]:
# check data types
final_salaries.dtypes

In [ ]:
# save file path
save_file_path = os.path.join(pwd,'salaries_1985to2022.csv')
final_salaries.to_csv(save_file_path, index = False)

In [ ]:
# connect to the AWS postres 
from sqlalchemy import create_engine
import psycopg2 
import io
from config import password

DB_address = 'nbadb.ca9dadq6ltaa.us-east-2.rds.amazonaws.com'
engine = create_engine(f'postgresql://team:{password}@{DB_address}:5432/NBA_database')
final_salaries.head(0).to_sql('salaries_1985to2022', engine, if_exists='replace',index=False) #drops old table and creates new empty table

conn = engine.raw_connection()
cur = conn.cursor()
output = io.StringIO()
final_salaries.to_csv(save_file_path, header= False, index = False)
output.seek(0)
contents = output.getvalue()
cur.copy_from(output, 'salaries_1985to2022', null="") # null values become ''
conn.commit()